In [25]:
import os
import random
import numpy as np
import pandas as pd
from collections import defaultdict
from sklearn.model_selection import train_test_split


import autograd
from autograd import grad
import autograd.numpy as anp  
from types import MethodType

from gensim.models.poincare import (
    PoincareKeyedVectors,
    PoincareModel,
    PoincareBatch
)


def no_op_compute_all(self):
    self.loss = 0.0

PoincareBatch.compute_all = no_op_compute_all



def anp_safe_norm(x, axis=None, eps=1e-9):
    return anp.sqrt(anp.sum(x * x, axis=axis) + eps)

def anp_safe_arccosh(x, eps=1e-9):
    clipped_x = anp.clip(x, 1.0 + eps, 1e15)
    inside = clipped_x * clipped_x - 1.0
    return anp.log(clipped_x + anp.sqrt(inside + eps))

def custom_loss_with_alpha(matrix, alpha, regularization_coeff=1.0):

    kappa = -anp.exp(alpha)
    vector_u = matrix[0]
    vectors_v = matrix[1:] 
    eucl_dist = anp_safe_norm(vector_u - vectors_v, axis=1)
    norm_u = anp_safe_norm(vector_u)
    norms_v = anp_safe_norm(vectors_v, axis=1)
    denom_u = (1.0 + kappa * (norm_u**2))
    denom_v = (1.0 + kappa * (norms_v**2))
    denom = denom_u * denom_v
    denom = anp.clip(denom, 1e-9, 1e15)

    gamma = 1.0 + 2.0*(eucl_dist**2) / denom
    gamma = anp.clip(gamma, 1.0 + 1e-9, 1e15)

    poinc_dist = anp_safe_arccosh(gamma)
    exp_neg = anp.exp(-poinc_dist)
    reg_term = regularization_coeff * anp.sum(vectors_v[0] * vectors_v[0])

    return -anp.log(exp_neg[0] / anp.sum(exp_neg)) + reg_term


_loss_grad_wrt_both = grad(custom_loss_with_alpha, argnum=[0, 1])



def monkey_patch_train_on_batch_with_alpha(self, relations, check_gradients=False):

    all_negatives = self._sample_negatives_batch(r[0] for r in relations)
    batch = self._prepare_training_batch(relations, all_negatives, check_gradients)

    total_loss = 0.0
    for (u, v), negs in zip(relations, all_negatives):
        vec_u = self.kv.vectors[u]
        vec_v = self.kv.vectors[v]
        vec_negs = self.kv.vectors[negs]
        sub_matrix = np.vstack((vec_u, vec_v, vec_negs))
        grad_matrix, grad_alpha = _loss_grad_wrt_both(
            sub_matrix, self.alpha, self.regularization_coeff
        )
        self.kv.vectors[u] -= self.alpha_emb * grad_matrix[0]
        self.kv.vectors[v] -= self.alpha_emb * grad_matrix[1]
        for i, neg_idx in enumerate(negs):
            self.kv.vectors[neg_idx] -= self.alpha_emb * grad_matrix[2 + i]
        self.kv.vectors[u] = self._clip_vectors(self.kv.vectors[u], self.epsilon)
        self.kv.vectors[v] = self._clip_vectors(self.kv.vectors[v], self.epsilon)
        self.kv.vectors[negs] = self._clip_vectors(self.kv.vectors[negs], self.epsilon)
        self.alpha -= self.lr_alpha * grad_alpha
        example_loss = float(custom_loss_with_alpha(sub_matrix, self.alpha, self.regularization_coeff))
        total_loss += example_loss

    batch.loss = total_loss / len(relations)
    return batch



def set_learnable_kappa_reparam(model, alpha_init=0.0, lr_alpha=1e-3, alpha_emb=0.1):

    model.alpha = alpha_init
    model.lr_alpha = lr_alpha
    model.alpha_emb = alpha_emb
    model._train_on_batch = MethodType(monkey_patch_train_on_batch_with_alpha, model)

    print(f"[INFO] reparam kappa => alpha_init={alpha_init}, lr_alpha={lr_alpha}, alpha_emb={alpha_emb}")


def curved_poincare_distance(u, v, alpha):
    kappa = -np.exp(alpha)  
    eucl_dist = np.linalg.norm(u - v)
    norm_u = np.linalg.norm(u)
    norm_v = np.linalg.norm(v)
    denom = (1.0 + kappa*(norm_u**2))*(1.0 + kappa*(norm_v**2))
    if denom <= 1e-15:
        denom = 1e-15
    gamma = 1.0 + 2.0*(eucl_dist**2)/denom
    if gamma < 1.0:
        gamma = 1.0
    return np.arccosh(gamma)

def custom_vector_distance(self, u, v):
    alpha = getattr(self, "model_alpha", 0.0)  
    return curved_poincare_distance(u, v, alpha)

def custom_vector_distance_batch(self, u, all_v):
    alpha = getattr(self, "model_alpha", 0.0)
    kappa = -np.exp(alpha)
    euc = np.linalg.norm(u - all_v, axis=1)
    norm_u = np.linalg.norm(u)
    norms_v = np.linalg.norm(all_v, axis=1)

    denom = (1.0 + kappa*(norm_u**2))*(1.0 + kappa*(norms_v**2))
    denom[denom < 1e-15] = 1e-15
    gamma = 1.0 + 2.0*(euc**2)/denom
    gamma[gamma < 1.0] = 1.0
    return np.arccosh(gamma)

PoincareKeyedVectors.vector_distance = custom_vector_distance
PoincareKeyedVectors.vector_distance_batch = custom_vector_distance_batch

def no_op_loss_fn(*args, **kwargs):
    return 0.0

PoincareModel._loss_fn = no_op_loss_fn



def prepare_custom_data(
    tsv_path,
    hypernym_col="Hypernym",
    hyponym_col="Hyponym",
    subset_size=10000,
    test_size=0.2,
    seed=42
):
    df = pd.read_csv(tsv_path, sep="\t", header=0, names=[hypernym_col, hyponym_col])
    relations_list = []
    for i, row in df.iterrows():
        hypernym = str(row[hypernym_col]).strip()
        hyponym = str(row[hyponym_col]).strip()
        if not hypernym or not hyponym or hypernym == "nan" or hyponym == "nan":
            continue
        relations_list.append((hyponym, hypernym))

    print(f"Total edges (before subsampling): {len(relations_list)}")

    if subset_size is not None and len(relations_list) > subset_size:
        random.seed(seed)
        relations_list = random.sample(relations_list, subset_size)
        print(f"Using a SUBSET of {subset_size} edges for faster testing.")

    train_relations, test_relations = train_test_split(
        relations_list, test_size=test_size, random_state=seed
    )
    print(f"Train relations: {len(train_relations)}")
    print(f"Test relations : {len(test_relations)}")

    combined_relations = train_relations + test_relations
    return train_relations, test_relations, combined_relations

def remove_edges(relations, removal_probability=0.1, seed=42):
    random.seed(seed)
    kept = []
    for (child, ancestor) in relations:
        if random.random() > removal_probability:
            kept.append((child, ancestor))
    return kept

def train_and_evaluate_poincare(
    train_relations,
    test_relations,
    combined_relations,
    embedding_dim=5,
    epochs=300,
    n_negatives_strict=500,
):
    from gensim.models.poincare import PoincareModel
    print(f"[INFO] Training Poincaré (dim={embedding_dim}, epochs={epochs}) ...")

    model = PoincareModel(
        train_data=train_relations,
        size=embedding_dim,
        negative=10,
        burn_in=10
    )

    set_learnable_kappa_reparam(model, alpha_init=0.0, lr_alpha=1e-3, alpha_emb=0.1)

    model.train(epochs=epochs)

    final_alpha = model.alpha
    final_kappa = -np.exp(final_alpha)
    print(f"[INFO] final alpha={final_alpha}, => kappa={final_kappa}")

    model.kv.model_alpha = final_alpha

    u_to_all_neighbors = defaultdict(set)
    for (child, ancestor) in combined_relations:
        u_to_all_neighbors[child].add(ancestor)

    vocab_nodes = set(model.kv.index_to_key)
    vocab_list = list(vocab_nodes)
    all_edges_set = set(combined_relations)

    def reconstruction_mean_rank_strict_sampled(model, edges, n_negatives=500, seed=1234):
        rng = np.random.default_rng(seed)
        ranks = []
        for (u, v) in edges:
            if (u not in model.kv) or (v not in model.kv):
                continue
            neighbors = u_to_all_neighbors[u]
            neg_candidates = []
            attempts = 0
            while len(neg_candidates) < n_negatives and attempts < 10000:
                candidate = rng.choice(vocab_list)
                if candidate not in (u, v) and (candidate not in neighbors):
                    neg_candidates.append(candidate)
                attempts += 1
            candidates = neg_candidates + [v]
            dists = [(c, model.kv.distance(u, c)) for c in candidates]
            sorted_nodes = [x[0] for x in sorted(dists, key=lambda x: x[1])]
            try:
                rank = sorted_nodes.index(v) + 1
                ranks.append(rank)
            except ValueError:
                pass
        return float(np.mean(ranks)) if ranks else 0.0

    def reconstruction_map_strict_sampled(model, edges, n_negatives=500, seed=1234):
        rng = np.random.default_rng(seed)
        reciprocal_ranks = []
        for (u, v) in edges:
            if (u not in model.kv) or (v not in model.kv):
                continue
            neighbors = u_to_all_neighbors[u]
            neg_candidates = []
            attempts = 0
            while len(neg_candidates) < n_negatives and attempts < 10000:
                candidate = rng.choice(vocab_list)
                if candidate not in (u, v) and (candidate not in neighbors):
                    neg_candidates.append(candidate)
                attempts += 1
            candidates = neg_candidates + [v]
            dists = [(c, model.kv.distance(u, c)) for c in candidates]
            sorted_nodes = [x[0] for x in sorted(dists, key=lambda x: x[1])]
            try:
                r = sorted_nodes.index(v) + 1
                reciprocal_ranks.append(1.0 / r)
            except ValueError:
                pass
        return float(np.mean(reciprocal_ranks)) if reciprocal_ranks else 0.0

    def link_prediction_mean_rank(model, edges, num_negatives=50):
        rng = np.random.default_rng(1234)
        ranks = []
        for (source, target) in edges:
            if (source not in model.kv) or (target not in model.kv):
                continue
            neg_candidates = []
            attempts = 0
            while len(neg_candidates) < num_negatives and attempts < 10000:
                candidate = rng.choice(vocab_list)
                if candidate != target and (source, candidate) not in all_edges_set:
                    neg_candidates.append(candidate)
                attempts += 1
            if not neg_candidates:
                continue
            candidates = neg_candidates + [target]
            dists = [model.kv.distance(source, c) for c in candidates]
            sorted_candidates = [c for _, c in sorted(zip(dists, candidates), key=lambda x: x[0])]
            rank = sorted_candidates.index(target) + 1
            ranks.append(rank)
        return float(np.mean(ranks)) if ranks else 0.0

    def link_prediction_map(model, edges, num_negatives=50):
        rng = np.random.default_rng(1234)
        reciprocal_ranks = []
        for (source, target) in edges:
            if (source not in model.kv) or (target not in model.kv):
                continue
            neg_candidates = []
            attempts = 0
            while len(neg_candidates) < num_negatives and attempts < 10000:
                candidate = rng.choice(vocab_list)
                if candidate != target and (source, candidate) not in all_edges_set:
                    neg_candidates.append(candidate)
                attempts += 1
            if not neg_candidates:
                continue
            candidates = neg_candidates + [target]
            dists = [model.kv.distance(source, c) for c in candidates]
            sorted_candidates = [c for _, c in sorted(zip(dists, candidates), key=lambda x: x[0])]
            rank = sorted_candidates.index(target) + 1
            reciprocal_ranks.append(1.0 / rank)
        return float(np.mean(reciprocal_ranks)) if reciprocal_ranks else 0.0

    def precision_at_k(model, edges, k=10, num_negatives=50):
        rng = np.random.default_rng(1234)
        hits = 0
        count = 0
        for (source, target) in edges:
            if (source not in model.kv) or (target not in model.kv):
                continue
            neg_candidates = []
            attempts = 0
            while len(neg_candidates) < num_negatives and attempts < 10000:
                candidate = rng.choice(vocab_list)
                if candidate != target and (source, candidate) not in all_edges_set:
                    neg_candidates.append(candidate)
                attempts += 1
            if not neg_candidates:
                continue
            candidates = neg_candidates + [target]
            dists = [model.kv.distance(source, c) for c in candidates]
            sorted_candidates = [c for _, c in sorted(zip(dists, candidates), key=lambda x: x[0])]
            top_k_nodes = sorted_candidates[:k]
            if target in top_k_nodes:
                hits += 1
            count += 1
        return hits / count if count else 0.0

    def recall_at_k(model, edges, k=10, num_negatives=50):
        return precision_at_k(model, edges, k, num_negatives)


    recon_mr_strict = reconstruction_mean_rank_strict_sampled(
        model, train_relations, n_negatives=n_negatives_strict, seed=42
    )
    recon_map_strict = reconstruction_map_strict_sampled(
        model, train_relations, n_negatives=n_negatives_strict, seed=42
    )

    lp_mr = link_prediction_mean_rank(model, test_relations, num_negatives=50)
    lp_map_ = link_prediction_map(model, test_relations, num_negatives=50)
    lp_p10 = precision_at_k(model, test_relations, k=10, num_negatives=50)
    lp_r10 = recall_at_k(model, test_relations, k=10, num_negatives=50)

    return {
        "reconstruction_mean_rank_strict": recon_mr_strict,
        "reconstruction_map_strict": recon_map_strict,
        "lp_mean_rank": lp_mr,
        "lp_map": lp_map_,
        "lp_precision_10": lp_p10,
        "lp_recall_10": lp_r10
    }

def run_multiple_experiments(
    address,
    dim_list,
    removal_prob_list,
    tsv_path,
    subset_size=10000,
    test_size=0.2,
    epochs=300,
    n_negatives_strict=500,
    seed=42
):
    train_rel_clean, test_rel, combined_rel = prepare_custom_data(
        tsv_path=tsv_path,
        hypernym_col="Hypernym",
        hyponym_col="Hyponym",
        subset_size=subset_size,
        test_size=test_size,
        seed=seed
    )

    all_results = []
    for embedding_dim in dim_list:
        for removal_prob in removal_prob_list:
            print("========================================================")
            print(f"RUNNING: dim={embedding_dim}, removal_prob={removal_prob:.2f}")
            print("========================================================")

           
            modified_train = remove_edges(train_rel_clean, removal_probability=removal_prob, seed=seed)
            removed_count = len(train_rel_clean) - len(modified_train)
            print(f"Removed {removed_count} edges from training set (prob={removal_prob}).")

            run_results = train_and_evaluate_poincare(
                train_relations=modified_train,
                test_relations=test_rel,
                combined_relations=combined_rel,
                embedding_dim=embedding_dim,
                epochs=epochs,
                n_negatives_strict=n_negatives_strict
            )

            row_info = {
                "embedding_dim": embedding_dim,
                "edge_removal_probability": removal_prob,
                "subset_size": subset_size,
                "test_size": test_size,
                "epochs": epochs,
                "n_negatives_strict": n_negatives_strict,
            }
            row_info.update(run_results)
            all_results.append(row_info)

    df = pd.DataFrame(all_results)
    df.to_excel(address, index=False)
    print(f"\nAll experiment results saved to: {address}")
    return df


if __name__ == "__main__":
    tsv_path = r"Y:\Data Science Readings\Geometry of Information\hebrew\hebewnet.tsv"
    dimension_list = [5, 10,20,50,100,200]
    removal_prob_list = [0.0, 0.1, 0.3] 
    excel_path = r"Y:\\Data Science Readings\\Geometry of Information\\Milestone 3\\Milestone3-HebrewNet\\HebrewnetM3exp.xlsx"

    final_df = run_multiple_experiments(
        address=excel_path,
        dim_list=dimension_list,
        removal_prob_list=removal_prob_list,
        tsv_path=tsv_path,
        subset_size=10000,
        test_size=0.2,
        epochs=100,
        n_negatives_strict=50,
        seed=42
    )
    print("\nFinal DataFrame of all runs:\n")
    print(final_df)


Total edges (before subsampling): 196
Train relations: 156
Test relations : 40
RUNNING: dim=5, removal_prob=0.00
Removed 0 edges from training set (prob=0.0).
[INFO] Training Poincaré (dim=5, epochs=100) ...
[INFO] reparam kappa => alpha_init=0.0, lr_alpha=0.001, alpha_emb=0.1
[INFO] final alpha=-3.7438519710219467, => kappa=-0.023662779016233124
RUNNING: dim=5, removal_prob=0.10
Removed 19 edges from training set (prob=0.1).
[INFO] Training Poincaré (dim=5, epochs=100) ...
[INFO] reparam kappa => alpha_init=0.0, lr_alpha=0.001, alpha_emb=0.1
[INFO] final alpha=-0.08607111079469706, => kappa=-0.9175289826851245
RUNNING: dim=5, removal_prob=0.30
Removed 54 edges from training set (prob=0.3).
[INFO] Training Poincaré (dim=5, epochs=100) ...
[INFO] reparam kappa => alpha_init=0.0, lr_alpha=0.001, alpha_emb=0.1
[INFO] final alpha=-0.1241894524662365, => kappa=-0.8832124982455102
RUNNING: dim=10, removal_prob=0.00
Removed 0 edges from training set (prob=0.0).
[INFO] Training Poincaré (dim=1